# RusTorch CoreML Integration - Python Bindings

This notebook demonstrates how to use RusTorch's CoreML functionality through Python bindings.

## Setup and Imports

In [ ]:
# Import RusTorch Python bindings
try:
    import rustorch
    print(f"✅ RusTorch version: {rustorch.__version__}")
    print(f"📝 Description: {rustorch.__description__}")
    print(f"👥 Author: {rustorch.__author__}")
except ImportError as e:
    print(f"❌ Failed to import RusTorch: {e}")
    print("Please build with maturin develop")
    exit(1)

import numpy as np
import platform

print(f"🖥️ Platform: {platform.system()} {platform.release()}")
print(f"🐍 Python version: {platform.python_version()}")

## Check CoreML Availability

In [ ]:
# Check CoreML functionality
try:
    # Check if CoreML is available
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML available: {coreml_available}")
    
    if coreml_available:
        print("🎉 CoreML is available!")
        
        # Get device information
        device_info = rustorch.get_coreml_device_info()
        print("📱 CoreML device information:")
        print(device_info)
    else:
        print("⚠️ CoreML is not available")
        if platform.system() != "Darwin":
            print("CoreML is only available on macOS")
        else:
            print("CoreML features may not be enabled")
            
except AttributeError:
    print("❌ CoreML functions not found")
    print("May not be built with CoreML features")
    coreml_available = False
except Exception as e:
    print(f"❌ Error checking CoreML: {e}")
    coreml_available = False

## CoreML Device Creation and Operations

In [ ]:
if coreml_available:
    try:
        # Create CoreML device
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ CoreML device created: {device}")
        
        # Get device information
        print(f"🆔 Device ID: {device.device_id()}")
        print(f"✅ Available: {device.is_available()}")
        print(f"💾 Memory limit: {device.memory_limit()} bytes")
        print(f"🧮 Compute units limit: {device.compute_units_limit()}")
        print(f"📚 Model cache size: {device.model_cache_size()}")
        
        # Cache cleanup
        device.cleanup_cache()
        print("🧹 Cache cleaned up")
        
    except Exception as e:
        print(f"❌ CoreML device operation error: {e}")
else:
    print("⚠️ Skipping device operations as CoreML is not available")

## CoreML Backend Configuration

In [ ]:
if coreml_available:
    try:
        # Create CoreML backend configuration
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ Backend config: {config}")
        
        # Check and modify configuration values
        print(f"📊 Enable caching: {config.enable_caching}")
        print(f"🗂️ Max cache size: {config.max_cache_size}")
        print(f"📈 Enable profiling: {config.enable_profiling}")
        print(f"🔄 Auto fallback: {config.auto_fallback}")
        
        # Modify configuration
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 Updated config: {config}")
        
        # Create CoreML backend
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 CoreML backend: {backend}")
        print(f"✅ Backend available: {backend.is_available()}")
        
        # Get backend statistics
        stats = backend.get_stats()
        print(f"📊 Backend stats: {stats}")
        print(f"   Total operations: {stats.total_operations}")
        print(f"   Cache hits: {stats.cache_hits}")
        print(f"   Cache misses: {stats.cache_misses}")
        print(f"   Fallback operations: {stats.fallback_operations}")
        print(f"   Cache hit rate: {stats.cache_hit_rate():.2%}")
        print(f"   Fallback rate: {stats.fallback_rate():.2%}")
        print(f"   Avg execution time: {stats.average_execution_time_ms:.2f}ms")
        
        # Cache cleanup
        backend.cleanup_cache()
        print("\n🧹 Backend cache cleaned")
        
    except Exception as e:
        print(f"❌ CoreML backend operation error: {e}")
else:
    print("⚠️ Skipping backend operations as CoreML is not available")

## Basic Tensor Operations (CPU)

For comparison with CoreML, let's first perform basic operations on CPU.

In [ ]:
try:
    # Basic tensor creation and operations
    print("🧮 Basic tensor operations (CPU)")
    
    # Create tensors from NumPy arrays (simplified interface)
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 Matrix A shape: {data_a.shape}")
    print(f"📐 Matrix B shape: {data_b.shape}")
    
    # Matrix multiplication with NumPy (for comparison)
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ NumPy matmul result shape: {numpy_result.shape}")
    print(f"📊 Result (first few elements): {numpy_result.flatten()[:4]}")
    
    print("\n🚀 CPU operations completed")
    
except Exception as e:
    print(f"❌ Tensor operation error: {e}")

## Performance Comparison Simulation

In [ ]:
import time

def benchmark_matrix_operations():
    """Compare performance with different matrix sizes"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 Performance comparison:")
    print("Size\t\tCPU Time (ms)\tExpected CoreML (ms)")
    print("-" * 50)
    
    for size in sizes:
        # Measure CPU execution time
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # Expected CoreML time (hypothetical)
        # In actual implementation, use real measurements from CoreML backend
        expected_coreml_time = cpu_time * 0.6  # Assumption: CoreML is 40% faster
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 Note: CoreML times are hypothetical. Actual values depend on specific implementation.")

## Device Selection Simulation

In [ ]:
def simulate_device_selection():
    """Simulate smart device selection"""
    
    operations = [
        ("Small matrix multiplication", (16, 16), "CPU"),
        ("Medium matrix multiplication", (128, 128), "Metal GPU"),
        ("Large matrix multiplication", (512, 512), "CoreML" if coreml_available else "Metal GPU"),
        ("Activation function", (32, 64, 128, 128), "Metal GPU"),
        ("Small convolution", (1, 3, 32, 32), "CPU"),
        ("Large convolution", (16, 64, 224, 224), "CoreML" if coreml_available else "Metal GPU"),
        ("Complex number operations", (128, 128), "Metal GPU"),  # CoreML not supported
        ("Statistical distribution", (1000,), "CPU"),  # CoreML not supported
    ]
    
    print("🎯 Smart device selection simulation:")
    print("Operation\t\t\tTensor Shape\t\tSelected Device")
    print("-" * 70)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<23}\t{shape_str:<15}\t{device}")
    
    print("\n📝 Selection logic:")
    print("  • Small operations: CPU (avoid overhead)")
    print("  • Medium operations: Metal GPU (balanced)")
    print("  • Large operations: CoreML (optimized)")
    print("  • Unsupported operations: GPU/CPU fallback")

simulate_device_selection()

## Practical Example: Simple Neural Network Layer

In [ ]:
def simulate_neural_network_layer():
    """Simulate neural network layer"""
    
    print("🧠 Neural network layer simulation:")
    
    # Batch size and layer configuration
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10 classes
    
    print(f"📊 Batch size: {batch_size}")
    print(f"🔢 Input features: {input_features}")
    print(f"🧮 Hidden features: {hidden_features}")
    print(f"🎯 Output features: {output_features}")
    
    # Forward pass simulation
    steps = [
        ("Input → Hidden", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("ReLU Activation", f"({batch_size}, {hidden_features})", "Metal"),
        ("Hidden → Output", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 Forward pass simulation:")
    total_time = 0
    
    for step, shape, device in steps:
        # Virtual execution time (ms)
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}ms")
    
    print(f"\n⏱️ Total forward pass time: {total_time:.2f}ms")
    print(f"🚀 Estimated throughput: {1000/total_time:.0f} batches/second")

simulate_neural_network_layer()

## Summary and Next Steps

In [ ]:
print("📋 RusTorch CoreML Integration Summary:")
print()
print("✅ Completed items:")
print("  • Jupyter environment setup")
print("  • Rust kernel and Python bindings creation")
print("  • CoreML availability check")
print("  • Device management and configuration")
print("  • Backend statistics and profiling")
print("  • Smart device selection")
print()
print("🚧 Future development:")
print("  • Actual CoreML operation implementation")
print("  • Performance benchmarking")
print("  • More activation functions and layer types")
print("  • Error handling improvements")
print("  • Memory optimization")
print()
print("🎯 Recommended next steps:")
print("  1. Load and test actual CoreML models")
print("  2. Compare Metal and CoreML performance")
print("  3. Test with real deep learning workflows")
print("  4. Evaluate in production environment")

if coreml_available:
    print("\n🎉 Congratulations! CoreML is available and all features can be tested.")
else:
    print("\n⚠️ CoreML is not available, but basic features are working.")
    print("   We recommend building with CoreML features enabled on macOS.")